In [4]:
import tensorflow as tf
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data', one_hot=True)
print("train size--", mnist.train.images.shape)
print("test size--", mnist.test.images.shape)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data\train-images-idx3-ubyte.gz


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data\train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data\t10k-images-idx3-ubyte.gz


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data\t10k-labels-idx1-ubyte.gz


train size-- (55000, 784)
test size-- (10000, 784)


In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 定义卷积层
def conv2d(x, W):
    #stride[0]表示输入channel数
    #stride[1]表示X方向步长
    #stride[2]表示Y方向步长
    #stride[3]表示输出kernel数
    #padding为'SAME'表示边界用0填充
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
# 定义池化层
def max_pool_2x2(x):
    #ksize表示池化窗口为2x2
    #strides表示步长为2
    #padding='VALID'表示边界信息抛弃
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='VALID')

# 输入数据占位符定义
X_input = tf.placeholder(tf.float32, [None, 784])
y_input = tf.placeholder(tf.float32, [None, 10])

X = tf.reshape(X_input, shape=[-1, 28, 28, 1])

# 卷积层1
# 窗口大小为5x5 输入channel数为1 输出kernel数为32
W_conv1 = weight_variable(shape=[5, 5, 1, 32])
b_conv1 = bias_variable(shape=[32])
h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

# 池化层1
# 输出为[None, 14, 14, 32]
h_pool1 = max_pool_2x2(h_conv1)

# 卷积层2
# 窗口的大小为5x5 输入channel数为32 输出kernel数为64
W_conv2 = weight_variable(shape=[5, 5, 32, 64])
b_conv2 = bias_variable(shape=[64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 池化层2
# 输出为[None, 7, 7, 64]
h_pool2 = max_pool_2x2(h_conv2)

# flatten为长向量
h_pool2_size = np.product([s.value for s in h_pool2.shape[1:]])
h_pool2_flat = tf.reshape(h_pool2, shape=[-1, h_pool2_size])

# 全连接层1
num_hidden1 = 256
W_fc1 = weight_variable(shape=[h_pool2_size, num_hidden1])
b_fc1 = bias_variable(shape=[num_hidden1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout : 随机失活
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层
num_class = 10
W_fc2 = weight_variable(shape=[num_hidden1, num_class])
b_fc2 = bias_variable(shape=[num_class])
y_pred = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    

In [9]:
print('X_input:', X_input)
print('X:      ', X)
print('y_input:', y_input)
print('h_conv1:', h_conv1)
print('h_pool1:', h_pool1)
print('h_conv2:', h_conv2)
print('h_pool2:', h_pool2)
print('h_pool2_flat', h_pool2_flat)
print('h_fc1:  ', h_fc1)
print('y_pred: ', y_pred)

X_input: Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
X:       Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
y_input: Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
h_conv1: Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
h_pool1: Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
h_conv2: Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
h_pool2: Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
h_pool2_flat Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
h_fc1:   Tensor("Relu_2:0", shape=(?, 256), dtype=float32)
y_pred:  Tensor("Softmax:0", shape=(?, 10), dtype=float32)
